# This file will serve as the main testing ground for a variety of signals for the CRSP data set

## The multiple strategies are the underlying:

### 1. Univariate Strategies
   * MACD (momentum), bollinger bands (mean reversion), univariate KRR
   
### 2. Multivariate Strategies
   * multivariate KRR, VAR
   
### 3. NN-based methodologies
   
### 4. Others? (eg. from academic literature)

### Further to do:

   * **ideas to include:**
       * PLS (pure dimension reduction but keeping objective), elastic net (as a way to include penalization methods)
       * momentum:
           * p.138 algo trading book: fama & blume 1966, Moskowitz Yao & Pedersen 2012 adaptation
       * pairs trading strategy using cointegration
       * mean reversion: individual (p.48 algo trading ernest chan) and in pairs (cointegrated assets: short one, long other, with and without bollinger bands, using kalman filters p.67,71,78)
       * seasonal trending strategy (buy yearly losers end of december and sell them end of january) -> Singal 2006
   * delve deeper into cross validation methodologies for each method, making sure all parameters are optimal
   * include fractional differentiation to maximise memory & keep stationarity
   * perform necessary statistical tests on VAR & its parameters
        * get VAR to work only with cointegrated pairs
   * generally: optimize code & parallelize operations

# Libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import statsmodels as sm
from sklearn.model_selection import *
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import *
from sklearn.kernel_ridge import *
from sklearn import linear_model
import math
import scipy
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import coint, adfuller
import statsmodels.api as sm
from statsmodels.tools.eval_measures import rmse, aic
from statsmodels.tsa.stattools import grangercausalitytests
from statsmodels.tsa.vector_ar.vecm import coint_johansen
from scipy.stats.mstats import winsorize
from scipy.stats import chi2
import warnings
warnings.filterwarnings("ignore")

# Preprocessing

In [3]:
data  = pd.read_csv('data/returns.csv')
data.set_index('date', inplace=True)
data.index = pd.to_datetime(data.index)
data = data.iloc[:,:20]

# Univariate strategies
* Simple strategies
    + mean reversion -> DONE
    + momentum (MACD) -> DONE
* forecasting strategies (requires cross-validation procedure)
    + ARMA
    + KRR (extremely slow) -> DONE

In [6]:
def momentum(ret):
    """
    This function is based on the Moskowitz et al. paper from 2012, whereby we go long for 1 month
    if the last 12 months were positive, in the other case, we go short.
    
    input: daily returns pandas dataframe of multivariate returns
    output: buy/sell signals dataframe
    """
    monthly_ret = ret.resample('M').sum()
    signal = pd.DataFrame(columns=monthly_ret.columns, index=monthly_ret.index)
        
    for m in range(13,monthly_ret.shape[0]):
        year = monthly_ret.iloc[(m-13):(m-1),:]
        yearly_ret = np.sort(np.array(year.sum(axis=0)))[::-1] # sort in descending order
        s = np.where(year.sum(axis=0) > 0, 1, 1)
        signal.iloc[m,:] = s
    
    signal = signal.replace(np.nan, 0)
    dates = ret.index
    dates.name = 'date'
    signal = signal.reindex(dates, method='bfill') # re-index to daily
    return signal
    

def macd(ret, long=8, short=4, signal_span=9):
    """
    calculates the MACD momentum strategy for a single time series
    input: pandas single series
    output: numpy array of signals
    """
    short_signal = ret.ewm(span=short, adjust=False).mean()
    long_signal = ret.ewm(span=long, adjust=False).mean() 
    macd = short_signal - long_signal
    
    pos = np.zeros(len(ret))
    pos = np.where(macd>0,1,-1)
    return pos

def macd_signals(returns,long=26,short=12,signal_span=9):
    """
    function calculating all the macd signals
    input: pandas dataframe of returns
    output: pandas dataframe of signals
    """
    signals = pd.DataFrame()
    for i in range(returns.shape[1]):
        signals['signal_{}'.format(i)] = macd(returns.iloc[:,i],long,short,signal_span)
    signals.index = returns.index
    return signals.shift(1).fillna(0)


# bollinger bands: mean reversion
def rev(ret,lookback,band,plot):
    """
    function with mean reversion bollinger bands
    """
    # work with prices
    p = (ret+1).cumprod()
    # obtain mean & vol
    m = p.rolling(window=lookback).mean()
    s = p.rolling(window=lookback).std()
    lower_band = m - band*s
    higher_band = m + band*s
    
    signal2 = np.zeros(len(ret))
    position_short = False
    position_long = False
    for i in range(len(ret)):
        # sell if higher than band
        if p[i] > higher_band[i]:
            signal2[i] = -1
            position_short = True
        # sell back if from lower band to mean
        elif p[i] < higher_band[i] and p.iloc[i] > m[i] and position_long == True:
            signal2[i] = 0
            position_long = False
        # buy back if from higher band to mean
        elif p[i] < m[i] and position_short == True:
            signal2[i] = 0
            position_short = False
        # buy if lower barrier hit
        elif p[i] < lower_band[i]:
            signal2[i] = 1
            position_long = True
        else:
            signal2[i] = signal2[i-1]
        
    return signal2


def reversion_signals(returns,lookback=100,band=2,plot=False):
    """
    calculates individual mean reversion based on bollinger bands
    """
    signals = pd.DataFrame()
    for i in range(returns.shape[1]):
        signals['signal_{}'.format(i)] = rev(ret = returns.iloc[:,i],lookback=lookback,band=band,plot=plot)
    signals.index = returns.index
    return signals.shift(1).fillna(0)

def pf_mean_reversion(returns, N=0.1):
    """
    Khandani & Lo (2011) : buy yesterday's N% worst losers & sell N% best winners
    """
    signals = pd.DataFrame(index=returns.index, columns=returns.columns)
    for i in range(returns.shape[1]):
        worst = returns.iloc[i,:].sort_values().index[:round(N*returns.shape[1])]
        worst_ind = [returns.columns.get_loc(col) for col in worst]
        best = returns.iloc[i,:].sort_values().index[-round(N*returns.shape[1]):]
        best_ind = [returns.columns.get_loc(col) for col in best]
        signals.iloc[i,worst_ind]  = 1
        signals.iloc[i,best_ind] = -1
    return signals.shift(1).fillna(0)

def cointegrated_pairs(series):
    """
    finds cointegrated pairs using engle-granger cointegration test for an input dataset
    """
    n = series.shape[1]
    keys = series.keys()
    pairs = []
    for i in range(n):
        for j in range(i+1, n):
            S1 = series[keys[i]]
            S2 = series[keys[j]]
            result = coint(S1, S2)
            pvalue = result[1]
            if pvalue < 0.025:
                pairs.append((keys[i], keys[j]))
    return pairs

def pairs_signals(returns, window=252):
    """
    pairs trading based on engle-granger cointegration test, signals are (inversely) proportional
    to the zscore of the ratio of cointegrated pairs
    """
    prices = (returns+1).cumprod()
    signals = pd.DataFrame(np.zeros(prices.shape), columns=prices.columns)
    for i in range(window,returns.shape[0],window):
        pairs = cointegrated_pairs(prices.iloc[i-252:i,:])
        for j in range(len(pairs)):
            p1 = prices.columns.get_loc(pairs[j][0])
            p2 = prices.columns.get_loc(pairs[j][1])
            
            # need 252 lookback for rolling mean
            S1 = prices.iloc[(i-252):min(i+252,prices.shape[0]), p1]
            S2 = prices.iloc[(i-252):min(i+252,prices.shape[0]), p2]
            ratio = S1 / S2 # faster way to do pairs trading
            
            # calculate rolling mean
            ratio_mean = ratio.rolling(252).mean()
            ratio_std = ratio.rolling(252).std()
            # technically mean should be backward looking -> data snooping here
            # correct this later
            # work with rolling
            zscores = (ratio[252:] - ratio_mean[252:])/ratio_std[252:]
            signals.iloc[i:min(i+252,prices.shape[0]),p1] = zscores
            signals.iloc[i:min(i+252,prices.shape[0]),p2] = -zscores
    return signals


propose to define a divergence
metric for each asset with the following formula β(ri,t −rf )−(rj,t −rf ) where β is the
regression coefficient of ri on rj , ri is the ith asset return and rj is the weighted average
of the 50 assets it correlates highest with. The top 10% are long and bottom 1% are
shorted. There is a one-month look-back and one-month holding period.

In [ ]:
def chen_divergence(returns):
    """
    signal construction based on Chen et al. (2019) quasi-multivariate pairs trading strategy
    """
    lookback = 3*returns.shape[1]
    c = returns.rolling(lookback).corr()
    signals = pd.DataFrame(np.zeros(returns.shape), columns=returns.columns)
    
    # loop : one-month look-back(or more like 3*number of columns) and one-month holding
    for i in range(lookback, returns.shape[0], lookback):
        c = returns.iloc[(i-lookback).]
    # regress each on top X assets (5, 10 or 20)
    # go long top 10% of predictions and short the bottom one
    

# Multivariate Econometric
   * Vector Autoregressive (VAR) -> DONE
   * Vector error correction models (VECM)
   * Kernel Ridge Regression (KRR) -> DONE

### Statistical tests
   * **Granger causality** (column var causes row var) -> rejection means good candidate for MV
       (tests the null hypothesis that the coefficients of past values in the regression equation is zero. In simpler terms, the past values of time series (X) do not cause the other series (Y))
   * **ADF test** -> need stationarity in time series
   * **cointegration** -> establish the presence of a statistically significant connection between two or more time series

In [ ]:
def grangers_causation_matrix(data, variables, test='ssr_chi2test', maxlag=4):    
    """
    data      : pandas dataframe containing the time series variables
    variables : list containing names of the time series variables.
    """
    df = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    for c in df.columns:
        for r in df.index:
            test_result = grangercausalitytests(data[[r, c]], maxlag=maxlag, verbose=False)
            p_values = [round(test_result[i+1][0][test][1],4) for i in range(maxlag)]
            min_p_value = np.min(p_values)
            df.loc[r, c] = min_p_value
    df.columns = [var + '_x' for var in variables]
    df.index = [var + '_y' for var in variables]
    return df

def adfuller_test(series, signif=0.05):
    """Perform ADFuller to test for Stationarity of given series of returns and print report"""
    for name, column in series.iteritems():
        r = adfuller(column, autolag='BIC')
        output = {'test_statistic':round(r[0], 4), 'pvalue':round(r[1], 4), 'n_lags':round(r[2], 4), 'n_obs':r[3]}
        p_value = output['pvalue']
        if p_value > signif:
            print("series {} is non-stationary".format(column.name))
            
def cointegration_test(df, alpha=0.05): 
    """Perform Johanson's Cointegration Test and Report Summary"""
    out = coint_johansen(df,-1,5)
    d = {'0.90':0, '0.95':1, '0.99':2}
    traces = out.lr1
    cvts = out.cvt[:, d[str(1-alpha)]]
    # print output if failed
    for name, trace, cvt in zip(df.columns, traces, cvts):
        if trace < cvt: print("column {} failed the cointegration test".format(name))

def multivariate_tests(data,alpha=0.05,maxlag=4, test = 'ssr_chi2test'):
    """
    data: pandas dataframe
    alpha: significance level
    maxlag: maximum number of lags for granger causality test
    test: test for granger causality
    
    what it does: prints failed adf and cointegration tests
    
    returns: granger causality matrix
    """
    adfuller_test(data, signif=alpha)
    cointegration_test(data, alpha=alpha)
    
    return grangers_causation_matrix(data, data.columns, test=test, maxlag=maxlag)

### VAR model
still requires some code optimisation

suggested improvements: build model using solely cointegrated time series -> can include more lags

In [ ]:
def var(df, window=30,maxlag=3,npredictions=5):
    
    n = len(df)
    # initiate forecasts dataframe
    forecasts = np.zeros((n+npredictions,df.shape[1])) 
    
    # rolling window approach
    for i in range(window,n,npredictions):
        if i % 250 == 0: print('{}% done'.format(round(i/(n-30)*100)),end="\r")
        bic_list = []
        series = df.iloc[(i-window):i,:]
        model = VAR(series)
        
        # optimal lags and fitting
        for j in np.arange(0,maxlag+1,1):
            try:
                result = model.fit(j)
                bic_list.append(result.bic)
            except:
                print("cannot deal with lag of {}, restarting with maxlag of {}".format(j,maxlag-1))
                if maxlag == 0:
                    print("window size too small relative to number of assets, please restart")
                    return 0
                else:
                    return var(df, window, maxlag-1)
        
        # final check
        if len(bic_list) == 1:
                print("window size too small relative to number of assets, please restart")
                return 0   
            
        nbr_lags = bic_list.index(min(bic_list)) + 1
        model_fitted = model.fit(nbr_lags)

        forecast_input = series.values[-nbr_lags:]
        
        fc = model_fitted.forecast(y=forecast_input, steps=npredictions)
        forecasts[i:(i+npredictions),:] = fc
        
    signals = np.where(forecasts > 0,1,-1)

                
    return signals

def krr_signals_mv(data,window_size=30,npredictions=5):
    n = len(data)

    predicted_values = np.zeros((n+npredictions,data.shape[1]))
    
    # define x and y
    x = data.shift(-1)
    y = data

    # rolling window, re-fit the model every "npredictions" days
    for i in range(window_size,n-npredictions,npredictions):
        if i % 100 == 0: print('{} % done'.format(round(i/(n-window_size)*100)),end="\r")
        begin = i - window_size
        
        # obtain principal components, section 2.5
        # question : is there no look-ahead bias by doing PCA on the same day returns, i.e. should it not be lagged?
        pca = PCA(n_components=4)
        principalComponents = pca.fit_transform(np.array(data.iloc[begin:i,:]))
        
        principalDf = pd.DataFrame(data = principalComponents, columns = ['pc1','pc2','pc3','pc4'])
        
        model = linear_model.LinearRegression()
        model_score = model.fit(principalDf,data.iloc[begin:i,:]).score(principalDf,data.iloc[begin:i,:])
        
        #x = np.array(data.iloc[(i-window_size):i,:])#.reshape(-1,1)
        #y = np.array(data.iloc[(i-window_size+1):(i+1),:]) 
        #x_pred = np.array(data.iloc[i:(i+npredictions),:])
        
        x_in = x.iloc[begin:i,:]
        y_in = y.iloc[begin:i,:]
        x_pred = x.iloc[i:(i+npredictions),:]
        
        # gaussian kernel parameters
        nbr_assets = data.shape[1]
        df = window_size - nbr_assets
        if df<0:df=1 # for chi-square
        r2 = np.array(model_score)
        sigma0 = math.sqrt(chi2.ppf(0.95,df)) / math.pi
        sigma = np.array([0.5*sigma0,sigma0,2*sigma0,4*sigma0,8*sigma0])
        lambda0 = (1-r2)/(r2)
        lambdaa = np.array([lambda0/8.0,lambda0/4.0,lambda0/2.0,lambda0,2*lambda0])
        
        parameters = {'kernel':['rbf'], 'alpha':lambdaa.tolist(),'degree':[2]}
        clf = GridSearchCV(KernelRidge(), parameters)
        clf = clf.fit(x_in,y_in)
        pred = clf.predict(x_pred)
        #print(i,i+npredictions)
        predicted_values[(i):(i+npredictions),:] = pred
    
    # turn values into signals
    signals = np.where(predicted_values > 0,1,-1)
    
    return signals

# NN-based methodologies
   * LSTM
   * CNN
   * other?

# (initial) Testing the strategies

Conclusions: long only is much better

In [ ]:
# performance evaluation
def pnl(data,signals):
    d = np.array(data)
    s = np.array(signals)
    return (pd.DataFrame(np.multiply(d,s), index=data.index)).cumsum()

In [ ]:
stocks = data
# 1. test macd
macd_multiple = macd_signals(stocks,26,9,9)
print("macd done")
# 2. test mean reversion
#m_r = reversion_signals(stocks)
print("mean reversion done")
# 3. test krr univariate
#krr_univ = krr_signals_univ(stocks)
print("krr univariate done")
# 4. test krr multivariate
#krr_mv = krr_signals_mv(stocks,window_size=15,npredictions=1)
print("krr multivariate done")
# 5. test VAR -> cannot deal with very high dimensions
#var_s = var(stocks.iloc[:,:5],window = 30,maxlag=3,npredictions=5)

In [ ]:
macd_perf = pnl(stocks,macd_multiple)
m_r_perf = pnl(stocks,m_r)
krr_perf = pnl(stocks,krr_mv[:-1,:])
#krr2_perf = pnl(stocks,krr_univ[:-5,:])
var_perf = pnl(stocks.iloc[:,:5],var_s[:-5,:])
momstr = pnl(data,momentum(data))

# skip first 30 days -> window size
macd_perf = macd_perf.iloc[30:,:]
m_r_perf = m_r_perf.iloc[30:,:]
krr_perf = krr_perf.iloc[30:,:]
#krr2_perf = krr2_perf.iloc[30:,:]
var_perf = var_perf.iloc[30:,:]
momstr = momstr.iloc[30:,:]
long_only = (stocks.iloc[30:,:]).cumsum()

In [ ]:
index = data.index[30:]
%matplotlib widget
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(index,macd_perf.mean(axis=1), label = 'macd', color = 'black')
ax.plot(index,momstr.mean(axis=1), label = 'momentum', color = 'r')

#ax2 = ax.twinx()
ax.plot(index,krr_perf.mean(axis=1), label = 'krr multivariate', color = 'b')
#ax.plot(index,krr2_perf.mean(axis=1), label = 'krr univariate', color = 'b')
ax.plot(index,m_r_perf.mean(axis=1), label = 'mean reversion', color = 'g')
ax.plot(index,long_only.mean(axis=1), label = 'real', color = 'y')
ax.plot(index,var_perf.mean(axis=1), label = 'var', color = 'gray')

ax.legend(loc=0)
#ax2.legend(loc=1)
ax.grid()
ax.set_xlabel("Time")
ax.set_ylabel("cumulative returns")
#ax2.set_ylabel("pred")
plt.show()